In [1]:
from heartnet_v1 import heartnet

In [2]:
load_path='weights.0148-0.8902.hdf5'

In [3]:
model = heartnet(load_path=load_path,FIR_train=False,trainable=False)

In [15]:
from keras.utils import plot_model
from keras.layers import Dense, Input

In [9]:
plot_model(model,'model.png')

In [8]:
dense_2 = model.layers.pop()

ValueError: Variable batch_normalization_1/moving_mean/biased already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py", line 1001, in moving_average_update
    x, value, momentum, zero_debias=True)
  File "/usr/local/lib/python2.7/dist-packages/keras/layers/normalization.py", line 185, in call
    self.momentum),
  File "/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py", line 617, in __call__
    output = self.call(inputs, **kwargs)
